In [1]:
import feedparser
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd

url = "http://news.google.com/news?q=covid&hl=en-US&sort=date&gl=US&num=10&output=rss"

class ParseFeed():
    def __init__(self, url):
        self.feed_url = url
        
    def clean(self, html):
        '''
        Get the text from html and do some cleaning
        '''
        soup = BeautifulSoup(html)
        text = soup.get_text()
        text = text.replace('\xa0', ' ')
        return text

    def parse(self):
        newsDescription = []
        newsUrl = []
        newsDate = []
        
        '''
        Parse the URL, and print all the details of the news 
        '''        
        feeds = feedparser.parse(self.feed_url).entries
        
        for f in feeds:
            newsDescription.append(self.clean(f.get("description", "")))
            newsUrl.append(f.get("link", ""))
            newsDate.append(f.get("published", ""))
            
        df = pd.DataFrame({'Description':newsDescription, 'URL':newsUrl, 'Date Published':newsDate})            
        return df
    
htmlOutput = ""
feed = ParseFeed(url)
df = feed.parse()

In [2]:
df['Date'] = pd.to_datetime(df['Date Published'])
df = df.head(10).sort_values(by = ['Date'], ascending=False)

# Saving data to html output
htmlOutput = """<html><title>COVID-19 Daily News</title><body><style>
		body {
		  background-image: url('background.jpg');
		  background-repeat: no-repeat;
		  background-attachment: fixed;
		  background-size: cover;
		}
	  </style><center><h1>COVID-19 Daily News</h1></center><dl>"""

for index,row in df.iterrows():
    htmlOutput = htmlOutput + "<dt>" + row['Description'] + "</dt><dd>" + row['Date Published'] + "</dd><dd><a href =" + row['URL'] + ">" + row['URL'] + "</a></dd><br>"
htmlOutput = htmlOutput + "</dl></body></html>"
text_file = open("home.html", "w")
text_file.write(htmlOutput)
text_file.close()

df.to_csv('data.csv', index=False)
df

,Description,URL,Date Published,Date
8,Live Covid-19 News Updates The New York Times,https://www.nytimes.com/live/2020/09/28/world/...,"Mon, 28 Sep 2020 19:40:22 GMT",2020-09-28 19:40:22+00:00
5,A guide to how — and when — a Covid-19 vaccine...,https://www.statnews.com/2020/09/28/a-layperso...,"Mon, 28 Sep 2020 19:07:30 GMT",2020-09-28 19:07:30+00:00
9,Michigan coronavirus (COVID-19) cases up to 12...,https://www.clickondetroit.com/health/2020/09/...,"Mon, 28 Sep 2020 18:58:00 GMT",2020-09-28 18:58:00+00:00
3,Global partnership to make available 120 milli...,https://www.who.int/news-room/detail/28-09-202...,"Mon, 28 Sep 2020 18:40:23 GMT",2020-09-28 18:40:23+00:00
1,Governor Cuomo Announces Moratorium on COVID-R...,https://www.governor.ny.gov/news/governor-cuom...,"Mon, 28 Sep 2020 16:53:47 GMT",2020-09-28 16:53:47+00:00
2,"MDH: 7 new COVID-19 deaths reported, state sur...",https://kstp.com/coronavirus/minnesota-departm...,"Mon, 28 Sep 2020 16:22:50 GMT",2020-09-28 16:22:50+00:00
6,Could Covid-19 be considered a preexisting con...,https://www.cnbc.com/2020/09/28/could-covid-19...,"Mon, 28 Sep 2020 15:31:00 GMT",2020-09-28 15:31:00+00:00
0,COVID-19 Daily Update 9-28-2020 West Virginia...,https://dhhr.wv.gov/News/2020/Pages/COVID-19-D...,"Mon, 28 Sep 2020 14:19:20 GMT",2020-09-28 14:19:20+00:00
4,Most US Households Struck By COVID-19 Suffer F...,https://www.npr.org/sections/health-shots/2020...,"Mon, 28 Sep 2020 10:00:00 GMT",2020-09-28 10:00:00+00:00
7,"COVID-19: Kidney Injury Links, Fewer Myocardia...",https://www.contagionlive.com/news/covid19-kid...,"Mon, 28 Sep 2020 09:02:41 GMT",2020-09-28 09:02:41+00:00
